In [ ]:
import pandas as pd

In [ ]:
alldf = pd.read_csv('starting-all-course-data.csv', index_col=False)
# alldf.drop(columns=['averageGPA', 'studentCount', 'W'], inplace=True)
alldf = alldf.rename(columns={"Number": "courseNo", "Course Title": "courseName", "Year": "year", "Term": "term", "Primary Instructor": "primaryInstructor", 
                              'A+': 'aPlus', 'A': 'a', 'A-': 'aMinus', 'B+': 'bPlus', 'B': 'b', 'B-': 'bMinus', 'C+': 'cPlus', 'C': 'c', 'C-': 'cMinus', 
                              'D+': 'dPlus', 'D': 'd', 'D-': 'dMinus', 'F': 'f'})
alldf = alldf[ alldf['term'] != 'Summer' ]
alldf.to_csv('starting-all-course-data.csv', index=False)
alldf

In [ ]:
csdf = alldf[ alldf['Subject'] == 'CS']
csdf.to_csv('starting-cs-course-data.csv', index=False)
csdf

In [ ]:
course_info = csdf[ ['courseNo', 'courseName', 'courseDesc'] ]
course_info = course_info.drop_duplicates(keep='first')
course_info.to_csv('cs-course-info.csv', index=False)
course_info

```
CREATE TABLE csCourse (
    courseNo INT UNSIGNED NOT NULL DEFAULT 0,
    courseName VARCHAR(500) DEFAULT "",
    courseDesc TEXT,
    PRIMARY KEY(courseNo, courseName)
);
```

In [ ]:
course_grades = csdf[ ['courseNo', 'courseName', 'year', 'term', 'primaryInstructor', 'aPlus', 'a', 'aMinus', 'bPlus', 'b', 'bMinus', 'cPlus', 'c', 'cMinus', 'dPlus', 'd', 'dMinus', 'f']]
course_grades.to_csv('cs-course-grades.csv', index=False)
course_grades

```
CREATE TABLE csGrade (
    courseNo INT UNSIGNED NOT NULL DEFAULT 0,
    courseName VARCHAR(500) NOT NULL DEFAULT "",
    year INT UNSIGNED NOT NULL DEFAULT 0,
    term VARCHAR(10) NOT NULL DEFAULT "",
    primaryInstructor VARCHAR(100) NOT NULL DEFAULT "",
    aPlus INT UNSIGNED NOT NULL DEFAULT 0,
    a INT UNSIGNED NOT NULL DEFAULT 0,
    aMinus INT UNSIGNED NOT NULL DEFAULT 0,
    bPlus INT UNSIGNED NOT NULL DEFAULT 0,
    b INT UNSIGNED NOT NULL DEFAULT 0,
    bMinus INT UNSIGNED NOT NULL DEFAULT 0,
    cPlus INT UNSIGNED NOT NULL DEFAULT 0,
    c INT UNSIGNED NOT NULL DEFAULT 0,
    cMinus INT UNSIGNED NOT NULL DEFAULT 0,
    dPlus INT UNSIGNED NOT NULL DEFAULT 0,
    d INT UNSIGNED NOT NULL DEFAULT 0,
    dMinus INT UNSIGNED NOT NULL DEFAULT 0,
    f INT UNSIGNED NOT NULL DEFAULT 0, 
    PRIMARY KEY(courseNo, courseName, year, term, primaryInstructor),
    FOREIGN KEY(courseNo) REFERENCES csCourse(courseNo)
    ON DELETE CASCADE
    ON UPDATE CASCADE
);
```

```
LOAD DATA LOCAL INFILE 'cs-course-grades.csv' 
INTO TABLE csGrade 
CHARACTER SET 'utf8mb4' 
FIELDS TERMINATED BY ','  
OPTIONALLY ENCLOSED BY '"' 
ESCAPED BY '\"'
LINES TERMINATED BY '\r\n' 
IGNORE 1 LINES;
```

```
LOAD DATA LOCAL INFILE 'cs-course-info.csv' 
INTO TABLE csCourse 
CHARACTER SET 'utf8mb4' 
FIELDS TERMINATED BY ','  
OPTIONALLY ENCLOSED BY '"' 
ESCAPED BY '\"' 
LINES TERMINATED BY '\r\n' 
IGNORE 1 LINES;
```

```
SELECT primaryInstructor, courseNo, courseName, (((SUM(aPlus) * 4) + (SUM(a) * 4) + (SUM(aMinus) * 3.67) + (SUM(bPlus) * 3.33) + (SUM(b) * 3) + (SUM(bMinus) * 2.67) + (SUM(cPlus) * 2.33) + (SUM(c) * 2) + (SUM(cMinus) * 1.67) + (SUM(dPlus) * 1.33) + (SUM(d) * 1) + (SUM(dMinus) * 0.67) + (SUM(f) * 0)) / (SUM(aPlus) + SUM(a) + SUM(aMinus) + SUM(bPlus) + SUM(b) + SUM(bMinus) + SUM(cPlus) + SUM(c) + SUM(cMinus) + SUM(dPlus) + SUM(d) + SUM(dMinus) + SUM(f))) as averageGPA
FROM csGrade
GROUP BY primaryInstructor, courseNo, courseName
ORDER BY courseNo;
```

```
SELECT primaryInstructor, courseNo, courseName, courseDesc
FROM csCourse NATURAL JOIN csGrade
GROUP BY primaryInstructor, courseNo, courseName
ORDER BY courseNo, primaryInstructor;
```